In [2]:
# from reranker import RerankerForInference
# rk = RerankerForInference.from_pretrained("results/checkpoint-140000/")  # load checkpoint


In [6]:
ls Reranker/src/reranker

__init__.py   arguments.py  dist/        trainer.py
__pycache__/  data.py       modeling.py


In [7]:
from Reranker.src.reranker.arguments import ModelArguments, DataArguments, \
    RerankerTrainingArguments as TrainingArguments

In [8]:
from transformers import AutoTokenizer



In [20]:
from reranker import Reranker 
from transformers import AutoTokenizer

In [24]:
from typing import Optional

import torch
import torch.functional as F
import copy
from transformers import AutoModelForSequenceClassification, AutoTokenizer,\
    PreTrainedModel, PreTrainedTokenizer, GPTNeoForSequenceClassification

from transformers.modeling_outputs import SequenceClassifierOutput, BaseModelOutputWithPooling
from torch import nn
import torch.distributed as dist

class Reranker(nn.Module):
    def __init__(
            self,
            hf_model: Optional[PreTrainedModel] = None,
            tokenizer: Optional[PreTrainedTokenizer] = None
    ):
        super().__init__()
        self.hf_model = hf_model
        self.tokenizer = tokenizer

    def tokenize(self, *args, **kwargs):
        return self.tokenizer(*args, **kwargs)

    def forward(self, batch):
        return self.hf_model(**batch)

    @classmethod
    def from_pretrained(cls, pretrained_model_name_or_path: str):
        hf_model = GPTNeoForSequenceClassification.from_pretrained(
            pretrained_model_name_or_path,num_labels=1)
        hf_tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neo-125M",    
                            bos_token="<|startoftext|>",
                            eos_token="<|endoftext|>",
                            pad_token="<|pad|>")
        
        hf_model.eval()
        return cls(hf_model, hf_tokenizer)

    def load_pretrained_model(self, pretrained_model_name_or_path, *model_args, **kwargs):
        self.hf_model = AutoModelForSequenceClassification.from_pretrained(
            pretrained_model_name_or_path, *model_args, **kwargs
        )

    def load_pretrained_tokenizer(self, pretrained_model_name_or_path, *inputs, **kwargs):
        self.tokenizer = AutoTokenizer.from_pretrained(
            pretrained_model_name_or_path, *inputs, **kwargs
        )

In [25]:
data_args ={
    "train_dir"
}

In [26]:
model = Reranker.from_pretrained("results/checkpoint-140000/") 

Some weights of GPTNeoForSequenceClassification were not initialized from the model checkpoint at results/checkpoint-140000/ and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [12]:
# rk.hf_model

In [150]:
each = json_c[999]
each

{'best_match': 'coe : list΄ pfun := λ n, (list',
 'score': 0.3333333333333333,
 'real': 'haveI := has_reflect',
 'all': ['exact (has_lift_',
  'exact (reflect l).symm',
  'assumption',
  'induction n with n IH',
  'induction l with',
  'convert to_sublist_of_reflect_nat_abs',
  'apply_instance',
  'induction l with n l ih',
  'coe : list΄ pfun := λ n, (list',
  'convert reflect_reflect l with t ht',
  "rw [mirror_eq_write' l]",
  'exact rintro ⟨_, rfl⟩',
  'rw ←@reflect.reflect tactic.rcases_patt',
  'exact (reflect_eq_',
  'convert reflection_reflect tactic.rcases_patt',
  'induction n with n l IH',
  'exact trans (mfld_trans_gen.mpr (reflect_reflect_reflect_modeq_right t ht)) rfl',
  'rw [← reflect_dvd_erson_aux, ← reflect_dvd_iff]',
  'exact rintro ⟨a, b, ⟨rfl⟩, rfl⟩',
  'exact (reflect_eq_self',
  'xt',
  'induction ih',
  'exact (reverse_rec_',
  'rcases this.exists_inv with ⟨R, hR⟩',
  'induction l with n l IH',
  'induction n with n l IH generalizing l',
  'exact (has_coe_to',
 

In [151]:
test = each["prompt"].replace("GOAL","<GOAL>").replace("PROOFSTEP","<PROOFSTEP>")

In [152]:
results = [i for i in each["all"]]

In [156]:
l = []
for i in results+[each["real"]]:
    inputs = rk.tokenize(test, i, return_tensors='pt')
    inputs.to(torch.device("cuda:0"))
    score = rk(inputs).logits
    l.append((score.cpu().detach().numpy()[0][1],i))
    l.sort(key=lambda x:x[0])

In [157]:
l

[(-2.0707211, 'rw [← reflect_dvd_erson_aux, ← reflect_dvd_iff]'),
 (-1.9039781, 'assumption'),
 (-1.7388834, 'rw [← forall_and_distrib, ←reflecting_C]'),
 (-1.6454703,
  'exact trans (mfld_trans_gen.mpr (reflect_reflect_reflect_modeq_right t ht)) rfl'),
 (-1.3055459, 'exact rintro ⟨_, rfl⟩'),
 (-1.235331, 'exact rintro ⟨a, b, ⟨rfl⟩, rfl⟩'),
 (-1.2266147, 'convert to_sublist_of_reflect_nat_abs'),
 (-1.1743462,
  'exact ⟨λ pos h1, forall_path h1, pos.reflect, diff h1.symm, diff pos.fst⟩'),
 (-1.0940642, "rw [mirror_eq_write' l]"),
 (-0.9648125, 'rcases this.exists_inv with ⟨R, hR⟩'),
 (-0.9581935, 'induction n with n l IH generalizing l'),
 (-0.92618155, 'rw ←@reflect.reflect tactic.rcases_patt'),
 (-0.82797396, 'exact (reflect_eq_self'),
 (-0.8064505, 'convert reflection_reflect tactic.rcases_patt'),
 (-0.43701363, 'induction n with n l IH'),
 (-0.38078067, 'induction l with n l IH'),
 (-0.29419905, 'exact (reflect l).symm'),
 (-0.24308807, 'induction n with n IH'),
 (-0.14588475, 'have

In [ ]:
"GOAL α : Type u,\t_inst_1 : inhabited α,\tb : buffer α,\ti : ℕ,\th : i < b.size\t⊢ b.read ⟨i, h⟩ = b.read i\n <PROOFSTEP> cases "

In [36]:
ls files_upload

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
data_test.jsonl           data_train_prepared.jsonl  wandb/
data_test_prepared.jsonl  data_valid.jsonl
data_train.jsonl          data_valid_prepared.jsonl


In [1]:
import json

with open('files_upload/data_train_prepared.jsonl', 'r') as json_file:
    json_c_all = [json.loads(i) for i in list(json_file)]

In [65]:
len(json_c_all)

168590

In [5]:
from transformers import AutoTokenizer

In [6]:

hf_tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neo-125M",    
                            bos_token="<|startoftext|>",
                            eos_token="<|endoftext|>",
                            pad_token="<|pad|>")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [7]:
from transformers import GPTNeoModel

In [8]:
data_args = {
    "train_dir":"data/train",
    "train_path":"data/train/train.json",
    "pred_dir":"data/dev",
    "pred_path":"data/train/dev.json",
    "train_group_size":8
}

In [9]:
from reranker.data import GroupedTrainDataset
train_dataset = GroupedTrainDataset(
    data_args, data_args["train_path"], 
    tokenizer=hf_tokenizer)

Using custom data configuration default-21c912046bcb6d4c
Reusing dataset json (/root/.cache/huggingface/datasets/json/default-21c912046bcb6d4c/0.0.0/ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b)


  0%|          | 0/1 [00:00<?, ?it/s]

In [11]:
# train_dataset[0]

In [17]:
from reranker import RerankerTrainer

In [18]:
from reranker.data import GroupedTrainDataset, PredictionDataset, GroupCollator

In [19]:
trainer = RerankerTrainer(
        model=model,
        train_dataset=train_dataset,
        data_collator=GroupCollator(hf_tokenizer)
    )

NameError: name 'model' is not defined

In [24]:
# training_args = TrainingArguments("test-trainer",fp16=True)

In [25]:
# training_args

In [26]:
# from transformers import TrainingArguments

In [13]:
trainer.train()

NameError: name 'trainer' is not defined

In [51]:
[i for i in hf_model(**inputs).last_hidden_state]

['last_hidden_state', 'past_key_values']

In [79]:
hf_model.cuda()

GPTNeoModel(
  (wte): Embedding(50259, 768)
  (wpe): Embedding(2048, 768)
  (drop): Dropout(p=0, inplace=False)
  (h): ModuleList(
    (0): GPTNeoBlock(
      (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (attn): GPTNeoAttention(
        (attention): GPTNeoSelfAttention(
          (attn_dropout): Dropout(p=0, inplace=False)
          (resid_dropout): Dropout(p=0, inplace=False)
          (k_proj): Linear(in_features=768, out_features=768, bias=False)
          (v_proj): Linear(in_features=768, out_features=768, bias=False)
          (q_proj): Linear(in_features=768, out_features=768, bias=False)
          (out_proj): Linear(in_features=768, out_features=768, bias=True)
        )
      )
      (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (mlp): GPTNeoMLP(
        (c_fc): Linear(in_features=768, out_features=3072, bias=True)
        (c_proj): Linear(in_features=3072, out_features=768, bias=True)
        (act): NewGELUActivation()
        (drop

In [81]:
json_c_all[0]

{'prompt': "GOAL α : Type u,\tb : buffer α,\ti : ℕ,\th : i < b.size,\tv : α\t⊢ b.write ⟨i, h⟩ v = b.write' i v\n PROOFSTEP ",
 'completion': " cases b; unfold write write'; simp [array.write_eq_write']\n"}

In [70]:
inputs.to(torch.device("cuda:0"))

{'input_ids': tensor([[11230,  1847,   371,  1058,  5994,   334,    11,   197,    43,  1058,
          5994,   410,    11,   197,    44,  1058,  5994,   266,    11,   197,
            62,  8625,    62,    16,  1058,   725,    62,  1806,   371,    11,
           197,    62,  8625,    62,    17,  1058,  6486,    62,  1806,   406,
            11,   197,    62,  8625,    62,    18,  1058,  6486,    62,   282,
         29230,   371,   406,    11,   197,    62,  8625,    62,    19,  1058,
           751,    62,  9503,    62,  8094,   337,    11,   197,    62,  8625,
            62,    20,  1058,  8265,   371,   337,    11,   197,    62,  8625,
            62,    21,  1058,  6486,    62,  1806,    62, 21412,   406,   337,
            11,   197,    62,  8625,    62,    22,  1058,  6486,    62, 21412,
           371,   406,   337,    11,   197,    45,   399,     6,  1058,  6486,
            62,  7266, 21412,   371,   406,   337,    11,   197,    71,  1058,
         24935,    45,   796, 24935,  

In [56]:
import numpy as np

In [80]:
%%time
inputs = rk.tokenize("GOAL R : Type u,\tL : Type v,\tM : Type w,\t_inst_1 : comm_ring R,\t_inst_2 : lie_ring L,\t_inst_3 : lie_algebra R L,\t_inst_4 : add_comm_group M,\t_inst_5 : module R M,\t_inst_6 : lie_ring_module L M,\t_inst_7 : lie_module R L M,\tN N' : lie_submodule R L M,\th : ↑N = ↑N',\tm : M\t⊢ m ∈ N ↔ m ∈ N'\n <PROOFSTEP> ", "rw [\u2190 mem_coe_submodule, h]\n", return_tensors='pt')
inputs.to(torch.device("cuda:0"))
torch.mean(hf_model(**inputs).last_hidden_state,axis=1)

CPU times: user 34.9 ms, sys: 20.5 ms, total: 55.3 ms
Wall time: 51.9 ms


tensor([[ 2.5791e-01,  2.2195e-01,  6.1754e-01, -4.1989e-01, -1.1693e+00,
         -4.2416e-01, -1.2921e-02, -6.3473e-02,  1.8195e-01, -2.2530e-02,
          6.7583e-01, -5.1123e-01,  8.9316e-02,  7.8981e-01, -2.5015e-01,
          1.9408e-01,  6.3822e-01,  9.7251e-01, -7.8701e-02, -6.0445e-01,
         -9.8325e-01,  2.0833e-01, -1.9024e+00,  9.8087e-01,  2.8801e-02,
         -5.0874e-01,  1.5393e+00, -3.0677e-01,  1.4771e-01, -1.6751e+00,
         -1.6980e+00,  8.8647e-01,  8.6847e-01,  8.0590e-01, -1.2548e+00,
         -1.8401e-02, -5.4263e-01, -6.5954e-01, -3.4339e-01, -4.9939e-01,
          5.0794e-01,  4.8216e-01,  6.5291e-02,  6.9481e-02,  9.3943e-01,
          1.1056e-01, -9.0834e-01,  5.9255e-01,  6.8692e-01,  2.9180e-01,
          3.6947e-01,  7.4422e-01,  1.4877e-01,  4.4374e-01,  1.0055e+00,
         -7.2418e-01,  1.5478e-01,  2.2642e-01,  2.0349e-01,  6.3065e-01,
         -4.8900e-01, -1.1986e+00, -2.5607e-01,  9.0008e-01,  1.2902e-01,
          5.3663e+00, -1.3429e-01,  7.

In [75]:
torch.cat((inputs,inputs))

TypeError: expected Tensor as element 0 in argument 0, but got BatchEncoding

In [76]:
hf_model

GPTNeoModel(
  (wte): Embedding(50259, 768)
  (wpe): Embedding(2048, 768)
  (drop): Dropout(p=0, inplace=False)
  (h): ModuleList(
    (0): GPTNeoBlock(
      (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (attn): GPTNeoAttention(
        (attention): GPTNeoSelfAttention(
          (attn_dropout): Dropout(p=0, inplace=False)
          (resid_dropout): Dropout(p=0, inplace=False)
          (k_proj): Linear(in_features=768, out_features=768, bias=False)
          (v_proj): Linear(in_features=768, out_features=768, bias=False)
          (q_proj): Linear(in_features=768, out_features=768, bias=False)
          (out_proj): Linear(in_features=768, out_features=768, bias=True)
        )
      )
      (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (mlp): GPTNeoMLP(
        (c_fc): Linear(in_features=768, out_features=3072, bias=True)
        (c_proj): Linear(in_features=3072, out_features=768, bias=True)
        (act): NewGELUActivation()
        (drop